In [433]:
import os

import random

import uuid

import pymysql
import redis
import rediscluster

import socket

from transliterate import translit

In [434]:
from IPython.display import display
from ipyfilechooser import FileChooser

## Задание 1

In [182]:
con = pymysql.connect(host='localhost',user='rayxxx',password='password')
print(con)
con.query('use nanoedu2015;')

0

In [183]:
dbName ='nanoedu2015'
viewName = 'sp_vuz_reg_city'

crs = con.cursor()
crs.execute(f'describe {dbName}.{viewName}')
viewDescribe = crs.fetchall()
for d in viewDescribe:
    print(d)

('id', 'int', 'NO', '', '0', '')
('short_name', 'varchar(500)', 'NO', '', None, '')
('sp_city_id', 'int', 'NO', '', None, '')
('checked', 'int', 'NO', '', '0', '')
('region_id', 'int', 'NO', '', '0', '')
('region_name', 'varchar(200)', 'YES', '', None, '')
('city_id', 'int', 'NO', '', '0', '')
('city_name', 'varchar(100)', 'YES', '', None, '')


In [423]:

crs.execute(f'select region_name, count(id) from {dbName}.{viewName} group by region_name')
data = crs.fetchall()

reg_vuz_count ={}
for row in data:
    reg_vuz_count[row[0]]=row[1]
  

In [432]:

for (k,v) in zip(reg_vuz_count.keys(),reg_vuz_count.values()):
    print(f'{k:<30} : {v:>5}')
    

Адыгея                         :     2
Алтай                          :     1
Алтайский край                 :     4
Амурская область               :     2
Архангельская область          :     1
Астраханская область           :     1
Башкортостан                   :     4
Белгородская область           :     1
Брянская область               :     3
Бурятия                        :     2
Владимирская область           :     1
Волгоградская область          :     4
Вологодская область            :     3
Воронежская область            :     7
Дагестан                       :     3
Еврейская АО                   :     1
Забайкальский край             :     1
Ивановская область             :     5
Ингушетия                      :     1
Иркутская область              :     7
Кабардино-Балкария             :     1
Калмыкия                       :     1
Калужская область              :     1
Камчатский край                :     1
Карачаево-Черкесия             :     2
Карелия                  

## Задание 2

In [6]:

# two way nested list

class Node:
    
    def __init__(self,data=''):
        self.data = data
        self.next=None
        self.prev=None

class twoWayNestedList:
    
    def __init__(self):
        self.head=None
        self.length = 0
    
    def append(self,line : str):
        newNode = Node(line)
        if self.head==None:
            self.head=newNode
        else:
            
            curr = self.head
            while curr.next!=None:
                curr = curr.next
            
            curr.next = newNode
            newNode.prev = curr
            
            self.length+=1
        
    def getNodeByInd(self,ind:int):
        
        curr = self.head
        for i in range(ind):
            
            if curr.next == None:
                print('have no such node')
                return 0
            
            curr = curr.next
        
        return curr
   
    def display(self):
        curr = self.head
        while curr!=None:
            print(curr.data)
            curr = curr.next

    def getData(self):
        data=[]
        curr = self.head
        while curr!=None:
            data.append(curr.data)
            curr = curr.next
        return data
    
nestedList = twoWayNestedList()

nestedList.append('123')
nestedList.append('qwe')

nestedList.display()

data = nestedList.getData()
print(data)

123
qwe
['123', 'qwe']


In [8]:

IP = '127.0.0.1'
PORT = 3000

#creating server socket
os.system('fuser -vn tcp ' + str(PORT))
os.system('sleep 5')
serverSock =  socket.socket(socket.AF_INET,socket.SOCK_STREAM)
serverSock.bind((IP,PORT))
serverSock.listen(5)

# creating client socket
clientSock =  socket.socket(socket.AF_INET,socket.SOCK_STREAM)
clientSock.connect((IP,PORT))

conn,addr = serverSock.accept()

while 1:
    inp = input('enter message: ')
    if inp=='0':
        break
    clientSock.send(inp.encode('utf-8'))
    
    reply = conn.recv(1024).decode('utf-8')
    
    nestedList.append(reply)

serverSock.close()
clientSock.close()
nestedList.display()


enter message:  new line
enter message:  another line
enter message:  third line
enter message:  qweasdzxc
enter message:  123qwe456rty
enter message:  last line
enter message:  0


123
qwe
new line
another line
third line
qweasdzxc
123qwe456rty
last line


## Задание 3

In [388]:

os.system('redis-server --port 3500 &')


0

208150:C 09 Apr 2022 16:57:59.292 # oO0OoO0OoO0Oo Redis is starting oO0OoO0OoO0Oo
208150:C 09 Apr 2022 16:57:59.292 # Redis version=5.0.7, bits=64, commit=00000000, modified=0, pid=208150, just started
208150:C 09 Apr 2022 16:57:59.292 # Configuration loaded


In [389]:

redcon = redis.Redis(host='localhost',port='3500',db=0, charset="utf-8")
redcon.flushdb()
print(redcon)


208150:M 09 Apr 2022 16:57:59.292 * Increased maximum number of open files to 10032 (it was originally set to 4096).
208150:M 09 Apr 2022 16:57:59.293 * Running mode=standalone, port=3500.
208150:M 09 Apr 2022 16:57:59.293 # Server initialized
208150:M 09 Apr 2022 16:57:59.293 # WARNING you have Transparent Huge Pages (THP) support enabled in your kernel. This will create latency and memory usage issues with Redis. To fix this issue run the command 'echo never > /sys/kernel/mm/transparent_hugepage/enabled' as root, and add it to your /etc/rc.local in order to retain the setting after a reboot. Redis must be restarted after THP is disabled.
208150:M 09 Apr 2022 16:57:59.294 * Ready to accept connections
Redis<ConnectionPool<Connection<host=localhost,port=3500,db=0>>>


### пункт 1

In [390]:

for (k,v) in zip(reg_vuz_count.keys(),reg_vuz_count.values()):
    redcon.set(translit(k,reversed=True),v)

print(redcon.keys())


[b'Chechnja', b'Permskij kraj', b"Tul'skaja oblast'", b"Orenburgskaja oblast'", b'Altajskij kraj', b'Chuvashija', b"Tomskaja oblast'", b"Stavropol'skij kraj", b"Brjanskaja oblast'", b"Nizhegorodskaja oblast'", b"Sverdlovskaja oblast'", b'Hanty-Mansijskij avt. okr.', b'Habarovskij kraj', b'Bashkortostan', b"Rjazanskaja oblast'", b'Karelija', b'Moskva', b"Jaroslavskaja oblast'", b"Tjumenskaja oblast'", b"Vladimirskaja oblast'", b"Volgogradskaja oblast'", b"Kurganskaja oblast'", b'Severnaja Osetija - Alanija', b'Kamchatskij kraj', b'Udmurtija', b"Kaluzhskaja oblast'", b"Pskovskaja oblast'", b"Vologodskaja oblast'", b"Kostromskaja oblast'", b'Komi', b"Ivanovskaja oblast'", b'Dagestan', b'Marij El', b"Belgorodskaja oblast'", b'Mordovija', b'Tyva', b"Kemerovskaja oblast'", b"Sahalinskaja oblast'", b'Krasnojarskij kraj', b"Sevastopol'", b'Krym', b'Evrejskaja AO', b"Orlovskaja oblast'", b"Lipetskaja oblast'", b"Cheljabinskaja oblast'", b"Kurskaja oblast'", b"Voronezhskaja oblast'", b'Sankt-Pet

### пункт 2

In [391]:

redcon = redis.Redis(host='localhost',port='3500',db=1, charset="utf-8")
redcon.flushdb()
print(redcon)


Redis<ConnectionPool<Connection<host=localhost,port=3500,db=1>>>


In [392]:

numOfHtab = 20
UUIDs= [uuid.uuid4().hex for i in range(numOfHtab)]

dictLen = len(reg_vuz_count.keys())
for (k,v) in zip(reg_vuz_count.keys(),reg_vuz_count.values()):
    redcon.hset(str(random.choice(UUIDs)),mapping={translit(k,reversed=True):v}) 


In [393]:

print(redcon.keys())


[b'2cb1c3d211bd4efdabc56747fb34bd30',
 b'c39f501e18f64cfc968884ce94fed666',
 b'976a26b691914e5cbae96e83e908e84e',
 b'9906e69b66bc4ab7abc377e4ede3b23f',
 b'5172322d94d84f0ca27ca7b165f1f977',
 b'58c986b15b7049839cbcb7f92f0b5e23',
 b'88ed20ae77534ea498198151983b2957',
 b'0bb3463a34884b0f815679df2bea2d51',
 b'4bed54cac2fd42f69bea265332920a0f',
 b'12e2654e35e746cdb3d17d3f23108896',
 b'1d958aacfc3b4146bc568546f34db153',
 b'c78461d1233844b692a4e479e41db195',
 b'90265b9d0be64037967089ec2d83a610',
 b'b8e41a3374c54ebca06efe1736564b9e',
 b'7b0bca22c2aa463eae968c78d1d1e35b',
 b'3cab5d02dda047bdbddb81d3a3e4d09f',
 b'e7792ab9be2e4140a3fb1daf93a8c317',
 b'7871e18b6c9343b5bba7295979c7e919',
 b'c05c1a88649740efb3ca9de54f585f99',
 b'f6793fd15ef74c30a5b0101533ef0579']

In [394]:

Hkeys = redcon.hkeys(redcon.keys()[0])

print(Hkeys)
print(redcon.hget(redcon.keys()[0],Hkeys[0]))


[b'Karelija']
b'2'


### пункт 3

In [395]:

redcon = redis.Redis(host='localhost',port='3500',db=2, charset="utf-8")
redcon.flushdb()
print(redcon)


Redis<ConnectionPool<Connection<host=localhost,port=3500,db=2>>>


In [396]:

key = 'nestedListData'
for d in nestedList.getData():
    redcon.rpush(key,d)

redcon.lrange(key,0,-1)
    

[b'123',
 b'qwe',
 b'new line',
 b'another line',
 b'third line',
 b'qweasdzxc',
 b'123qwe456rty',
 b'last line']

## Задание 4

### Перенос баз данных, созданных в задании 3, на redis-cluster

In [397]:

redclust = rediscluster.RedisCluster(host='localhost',port='8000')
print(redclust)


RedisCluster<localhost:8000>


In [398]:
redclust.flushall()

{'127.0.0.1:8001': True, '127.0.0.1:8002': True, '127.0.0.1:8000': True}

### Пункт 1: запись данных пункта 1 задания 3

In [399]:

redcon = redis.Redis(host='localhost',port='3500',db=0, charset="utf-8")
print(redcon)


Redis<ConnectionPool<Connection<host=localhost,port=3500,db=0>>>


In [400]:

dbHTabName = 'region_vuz_number'

redclust.delete(dbHTabName)

keys = redcon.keys()
vals = [redcon.get(k) for k in keys]


for (k,v) in zip(reg_vuz_count.keys(),reg_vuz_count.values()):
    redclust.hset(dbHTabName,mapping={translit(k,reversed=True):v}) 


In [401]:
Keys = redclust.keys()
print(Keys[:min(5,len(Hkeys))])

Hkeys = redclust.hkeys(dbHTabName)
print(Hkeys[:min(5,len(Hkeys))])

print(redclust.hget(dbHTabName,Hkeys[0]))


[b'region_vuz_number']
[b'Adygeja', b'Altaj', b'Altajskij kraj', b"Amurskaja oblast'", b"Arhangel'skaja oblast'"]
b'2'


### Пункт 2: запись данных пункта 2 задания 3

In [402]:

redcon = redis.Redis(host='localhost',port='3500',db=1, charset="utf-8")
print(redcon)


Redis<ConnectionPool<Connection<host=localhost,port=3500,db=1>>>


In [403]:

dbListName = 'region_vuz_number_2'

redclust.delete(dbListName)

htabkeys = redcon.keys()
for tabk in htabkeys:
    for hk in redcon.hkeys(tabk):
        k = hk
        v = redcon.hget(tabk,hk)

        redclust.hset(tabk,mapping={k:v}) 
    redclust.rpush(dbListName,tabk) 
    

In [404]:

Keys = redclust.lrange(dbListName,0,-1)
print(Keys[0:min(5,len(Keys))])

tabKeys = redclust.hkeys(Keys[0])
print(tabKeys[0:min(5,len(tabKeys))])

print( redclust.hget(Keys[0],tabKeys[0]))


[b'2cb1c3d211bd4efdabc56747fb34bd30', b'c39f501e18f64cfc968884ce94fed666', b'976a26b691914e5cbae96e83e908e84e', b'9906e69b66bc4ab7abc377e4ede3b23f', b'5172322d94d84f0ca27ca7b165f1f977']
[b'Karelija']
b'2'


### Пункт 3: запись данных пункта 3 задания 3

In [405]:

redcon = redis.Redis(host='localhost',port='3500',db=2, charset="utf-8")
print(redcon)


Redis<ConnectionPool<Connection<host=localhost,port=3500,db=2>>>


In [406]:

ListName = redcon.keys()[0]
redclust.delete(ListName)

records = redcon.lrange(ListName,0,-1)

print(ListName)
print(records)

for r in records:
    redclust.rpush(ListName,r)


b'nestedListData'
[b'123', b'qwe', b'new line', b'another line', b'third line', b'qweasdzxc', b'123qwe456rty', b'last line']


In [407]:

records = redclust.lrange(ListName,0,-1)
print(records)


[b'123', b'qwe', b'new line', b'another line', b'third line', b'qweasdzxc', b'123qwe456rty', b'last line']


### Перенос данных из redis-cluster обратно в mysql

In [408]:

redclust = rediscluster.RedisCluster(host='localhost',port='8000')
print(redclust)


RedisCluster<localhost:8000>


In [409]:

sqlconn = pymysql.connect(host='localhost',user='root',password='password');
sqlconn.query('use Redis;')
print(sqlconn)


In [410]:

crs = sqlconn.cursor()

crs.execute('show tables;')
tablesList = crs.fetchall()

print(tablesList)


(('nested_list_records',), ('region_vuz_number',), ('region_vuz_number_2',))


### Пункт 1: запись данных пункта 1 задания 4

In [411]:

tabName = 'region_vuz_number'
crs.execute(f'truncate {tabName};')


0

In [412]:

crs.execute(f'describe region_vuz_number;')
tab1Describe = crs.fetchall()
for line in tab1Describe:
    print(line)


('region_name', 'varchar(40)', 'NO', 'PRI', None, '')
('vuz_number', 'int', 'YES', '', '0', '')


In [413]:

keys = redclust.hkeys(tabName)
vals = [redclust.hget(tabName,k) for k in keys]

for (k,v) in zip(keys,vals):
    query = f'insert into region_vuz_number(region_name,vuz_number) values ("{k.decode("utf-8")[:-1]}",{v.decode("utf-8")})'
    crs.execute(query)


In [414]:

crs.execute('select * from region_vuz_number')
data = crs.fetchall()
print(data)


(('Adygej', 2), ('Alta', 1), ('Altajskij kra', 4), ('Amurskaja oblast', 2), ("Arhangel'skaja oblast", 1), ('Astrahanskaja oblast', 1), ('Bashkortosta', 4), ('Belgorodskaja oblast', 1), ('Brjanskaja oblast', 3), ('Burjatij', 2), ('Chechnj', 3), ('Cheljabinskaja oblast', 3), ('Chuvashij', 2), ('Dagesta', 3), ('Evrejskaja A', 1), ('Habarovskij kra', 5), ('Hakasij', 1), ('Hanty-Mansijskij avt. okr', 1), ('Ingushetij', 1), ('Irkutskaja oblast', 7), ('Ivanovskaja oblast', 5), ('Jaroslavskaja oblast', 4), ('Kabardino-Balkarij', 1), ('Kalmykij', 1), ('Kaluzhskaja oblast', 1), ('Kamchatskij kra', 1), ('Karachaevo-Cherkesij', 2), ('Karelij', 2), ('Kemerovskaja oblast', 5), ('Kirovskaja oblast', 1), ('Kom', 2), ('Kostromskaja oblast', 2), ('Krasnodarskij kra', 4), ('Krasnojarskij kra', 5), ('Kry', 1), ('Kurganskaja oblast', 2), ('Kurskaja oblast', 2), ('Lipetskaja oblast', 3), ('Magadanskaja oblast', 1), ('Marij E', 1), ('Mordovij', 2), ('Moskovskaja oblast', 4), ('Moskv', 35), ('Murmanskaja obla

### Пункт 2: запись данных пункта 2 задания 4

In [415]:

tabName = 'region_vuz_number_2'
crs.execute(f'truncate {tabName};')


0

In [416]:

crs.execute(f'describe {tabName};')
tab1Describe = crs.fetchall()
for line in tab1Describe:
    print(line)


('region_name', 'varchar(40)', 'NO', 'PRI', None, '')
('vuz_number', 'int', 'YES', '', '0', '')


In [417]:

ListKeys = redclust.lrange(dbListName,0,-1)
for tab in ListKeys:
    for tk in redclust.hkeys(tab):
        k = tk
        v = redclust.hget(tab,tk)
        query = f'insert into {tabName}(region_name,vuz_number) values ("{k.decode("utf-8")[:-1]}",{v.decode("utf-8")})'
        crs.execute(query)


In [418]:

crs.execute('select * from region_vuz_number_2')
data = crs.fetchall()
print(data)


(('Adygej', 2), ('Alta', 1), ('Altajskij kra', 4), ('Amurskaja oblast', 2), ("Arhangel'skaja oblast", 1), ('Astrahanskaja oblast', 1), ('Bashkortosta', 4), ('Belgorodskaja oblast', 1), ('Brjanskaja oblast', 3), ('Burjatij', 2), ('Chechnj', 3), ('Cheljabinskaja oblast', 3), ('Chuvashij', 2), ('Dagesta', 3), ('Evrejskaja A', 1), ('Habarovskij kra', 5), ('Hakasij', 1), ('Hanty-Mansijskij avt. okr', 1), ('Ingushetij', 1), ('Irkutskaja oblast', 7), ('Ivanovskaja oblast', 5), ('Jaroslavskaja oblast', 4), ('Kabardino-Balkarij', 1), ('Kalmykij', 1), ('Kaluzhskaja oblast', 1), ('Kamchatskij kra', 1), ('Karachaevo-Cherkesij', 2), ('Karelij', 2), ('Kemerovskaja oblast', 5), ('Kirovskaja oblast', 1), ('Kom', 2), ('Kostromskaja oblast', 2), ('Krasnodarskij kra', 4), ('Krasnojarskij kra', 5), ('Kry', 1), ('Kurganskaja oblast', 2), ('Kurskaja oblast', 2), ('Lipetskaja oblast', 3), ('Magadanskaja oblast', 1), ('Marij E', 1), ('Mordovij', 2), ('Moskovskaja oblast', 4), ('Moskv', 35), ('Murmanskaja obla

### Пункт 3: запись данных пункта 3 задания 4

In [441]:

tabName = 'nested_list_records'
crs.execute(f'truncate {tabName};')


0

In [442]:

crs.execute(f'describe {tabName};')
tab1Describe = crs.fetchall()
for line in tab1Describe:
    print(line)


('id', 'mediumint', 'NO', 'PRI', None, 'auto_increment')
('record', 'varchar(255)', 'YES', '', 'empty', '')


In [443]:

for rec in redclust.lrange('nestedListData',0,-1):
    query = f"insert into {tabName}(record) values ('{rec.decode('utf-8')}')"
    crs.execute(query)


In [444]:

crs.execute(f'select * from {tabName}')
data = crs.fetchall()
print(data)


((1, '123'), (2, 'qwe'), (3, 'new line'), (4, 'another line'), (5, 'third line'), (6, 'qweasdzxc'), (7, '123qwe456rty'), (8, 'last line'))
